In [1]:
import json
from collections import defaultdict
import pandas as pd
import numpy as np
from tqdm import tqdm
from scipy.spatial import cKDTree
from itertools import combinations
from haversine import haversine

In [2]:
# Load data from the CSV file into a new DataFrame
data = pd.read_csv('./model/gps_data_geolife.csv')

In [3]:
data = data[data['Latitude'] >= 39.75]
data = data[data['Latitude'] <= 40.1]
data = data[data['Longitude'] >= 116.18]
data = data[data['Longitude'] <= 116.6]

In [4]:
def calculate_distance(coord1, coord2):
	return haversine(coord1, coord2)

In [5]:
meeting_points = defaultdict(list)
meeting_distance_threshold = 50

for index, row in tqdm(data.iterrows(), desc="main data"):
	taxi_id = row["Person ID"]
	location = (row["Latitude"], row["Longitude"])

	if meeting_points[location]:
		for other_taxi_id in meeting_points[location]:
			if taxi_id != other_taxi_id:
				other_location = (data.loc[data["Person ID"] == other_taxi_id, "Latitude"].iloc[0], data.loc[data["Person ID"] == other_taxi_id, "Longitude"].iloc[0])

				distance = calculate_distance(location, other_location)
				if distance <= meeting_distance_threshold:
					meeting_info = {
						"Taxi1": int(other_taxi_id),
						"Taxi2": int(taxi_id),
						"Location": {
							"Latitude": location[0],
							"Longitude": location[1]
						},
						"Distance (meters)": distance,
						"Timestamp1": row["Timestamp"],
						"Timestamp2": data.loc[data["Person ID"] == other_taxi_id, "Timestamp"].iloc[0]
					}

					with open("./files/meeting_data_geolife2.json", "a") as json_file:
						json.dump(meeting_info, json_file)
						json_file.write("\n")
						
	meeting_points[location].append(taxi_id)

main data: 0it [00:00, ?it/s]

main data: 813395it [1:18:02, 173.72it/s]


KeyboardInterrupt: 